# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


In [ ]:
# YOUR CHANGES HERE
import pandas as pd

strawberry_data = pd.read_csv('strawberry-prices.tsv', sep='\t')
strawberry_data.head()



,month,price
0,2020-01-01,4.0490
1,2020-02-01,3.6250
2,2020-03-01,3.3770
3,2020-04-01,3.2328
4,2020-05-01,3.1260


In [ ]:
import numpy as np

strawberry_data['month'] = pd.to_datetime(strawberry_data['month'])
strawberry_data = strawberry_data.sort_values('month')

train = strawberry_data[strawberry_data['month'].dt.year <= 2023]
test_dates = strawberry_data[strawberry_data['month'].dt.year == 2024]['month']

monthly_avg = train.groupby(train['month'].dt.month)['price'].mean()

predictions = []
for date in test_dates:
    month_num = date.month
    pred_price = monthly_avg[month_num]
    predictions.append(pred_price)

backtest_df = pd.DataFrame({
    'month': test_dates.dt.strftime('%Y-%m-%d'),
    'price': predictions
})

# Save to file
backtest_df.to_csv('strawberry-backtest.tsv', sep='\t', index=False, float_format='%.4f')

print("Backtest predictions:")
print(backtest_df)

Backtest predictions:
         month     price
48  2024-01-01  4.362750
49  2024-02-01  4.091250
50  2024-03-01  3.688750
51  2024-04-01  3.787200
52  2024-05-01  3.526250
53  2024-06-01  3.275000
54  2024-07-01  3.194000
55  2024-08-01  3.489750
56  2024-09-01  3.581750
57  2024-10-01  3.926000
58  2024-11-01  4.398000
59  2024-12-01  4.788575


Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [6]:
# YOUR CHANGES HERE

df = strawberry_data.copy()
df['month'] = pd.to_datetime(df['month'])

backtest = pd.read_csv('strawberry-backtest.tsv', sep='\t')
backtest['month'] = pd.to_datetime(backtest['month'])

actual_2024 = df[df['month'].dt.year == 2024]

merged = pd.merge(actual_2024, backtest, on='month', suffixes=('_actual', '_predicted'))

residuals = merged['price_actual'] - merged['price_predicted']

mean_residual = residuals.mean()
std_residual = residuals.std()

accuracy_df = pd.DataFrame({
    'mean': [mean_residual],
    'std': [std_residual]
})

accuracy_df.to_csv('backtest-accuracy.tsv', sep='\t', index=False, float_format='%.6f')
accuracy_df

,mean,std
0,-0.004858,0.274557


Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [7]:
# YOUR CHANGES HERE

df = strawberry_data.copy()
df['month'] = pd.to_datetime(df['month'])
df = df.sort_values('month')

train = df[(df['month'].dt.year >= 2020) & (df['month'].dt.year <= 2024)]

monthly_avg = train.groupby(train['month'].dt.month)['price'].mean()

forecast_dates = pd.date_range(start='2025-01-01', end='2025-12-01', freq='MS')

predictions = []
for date in forecast_dates:
    month_num = date.month
    pred_price = monthly_avg[month_num]
    predictions.append(pred_price)

forecast_df = pd.DataFrame({
    'month': forecast_dates.strftime('%Y-%m-%d'),
    'price': predictions
})

forecast_df.to_csv('strawberry-forecast.tsv', sep='\t', index=False, float_format='%.4f')
forecast_df

,month,price
0,2025-01-01,4.50120
1,2025-02-01,4.12580
2,2025-03-01,3.69940
3,2025-04-01,3.74496
4,2025-05-01,3.46840
5,2025-06-01,3.21540
6,2025-07-01,3.17840
7,2025-08-01,3.46120
8,2025-09-01,3.61380
9,2025-10-01,3.88440


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [8]:
# YOUR CHANGES HERE

forecast = pd.read_csv('strawberry-forecast.tsv', sep='\t')
forecast['month'] = pd.to_datetime(forecast['month'])

INVESTMENT = 1000000  
FREEZE_COST = 0.20    
STORAGE_COST = 0.10   
DISCOUNT = 0.9       

results = []

for buy_idx, buy_row in forecast.iterrows():
    buy_month = buy_row['month']
    buy_price = buy_row['price']
    
    for sell_idx, sell_row in forecast.iterrows():
        sell_month = sell_row['month']
        sell_price = sell_row['price']
        
        if sell_month >= buy_month:
            months_stored = (sell_month.year - buy_month.year) * 12 + (sell_month.month - buy_month.month)
            
            total_cost_per_pint = buy_price + FREEZE_COST + (STORAGE_COST * months_stored)
            
            if months_stored > 0:  
                revenue_per_pint = sell_price * DISCOUNT
            else:  
                revenue_per_pint = sell_price
                total_cost_per_pint = buy_price  

            profit_per_pint = revenue_per_pint - total_cost_per_pint
            
            if total_cost_per_pint > 0:
                pints_purchased = INVESTMENT / total_cost_per_pint
                total_profit = profit_per_pint * pints_purchased
            else:
                pints_purchased = 0
                total_profit = 0
            
            results.append({
                'buy_month': buy_month.strftime('%Y-%m'),
                'sell_month': sell_month.strftime('%Y-%m'), 
                'pints_purchased': pints_purchased,
                'expected_profit': total_profit
            })

results_df = pd.DataFrame(results)

results_df = results_df.sort_values('expected_profit', ascending=False)

results_df.to_csv('timings.tsv', sep='\t', index=False, float_format='%.2f')

results_df


,buy_month,sell_month,pints_purchased,expected_profit
62,2025-07,2025-12,257838.283828,114395.111386
56,2025-06,2025-12,249041.191413,76373.462171
67,2025-08,2025-12,246232.640599,64234.708953
71,2025-09,2025-12,243084.253002,50627.157373
61,2025-07,2025-11,264662.290917,48634.342579
...,...,...,...,...
17,2025-02,2025-07,207219.528368,-407236.105931
8,2025-01,2025-09,181778.521050,-408779.902567
7,2025-01,2025-08,185144.042065,-423261.497445
5,2025-01,2025-06,192263.323848,-443616.857648


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [9]:
# YOUR CHANGES HERE


timings = pd.read_csv('timings.tsv', sep='\t')

accuracy = pd.read_csv('backtest-accuracy.tsv', sep='\t')
std_deviation = accuracy['std'].iloc[0]

best_scenario = timings.iloc[0]  
best_profit = best_scenario['expected_profit']
buy_month = best_scenario['buy_month']
sell_month = best_scenario['sell_month']
pints_purchased = best_scenario['pints_purchased']


forecast = pd.read_csv('strawberry-forecast.tsv', sep='\t')
forecast['month'] = pd.to_datetime(forecast['month'])
forecast['month_str'] = forecast['month'].dt.strftime('%Y-%m')

original_sell_price = forecast[forecast['month_str'] == sell_month]['price'].iloc[0]

sell_price_minus_std = original_sell_price - std_deviation


buy_price = forecast[forecast['month_str'] == buy_month]['price'].iloc[0]

buy_date = pd.to_datetime(buy_month + '-01')
sell_date = pd.to_datetime(sell_month + '-01')
months_stored = (sell_date.year - buy_date.year) * 12 + (sell_date.month - buy_date.month)

# Constants
FREEZE_COST = 0.20
STORAGE_COST = 0.10
DISCOUNT = 0.9
INVESTMENT = 1000000

if months_stored > 0:
    total_cost_per_pint = buy_price + FREEZE_COST + (STORAGE_COST * months_stored)
    revenue_per_pint_adjusted = sell_price_minus_std * DISCOUNT
else:
    total_cost_per_pint = buy_price
    revenue_per_pint_adjusted = sell_price_minus_std

profit_per_pint_adjusted = revenue_per_pint_adjusted - total_cost_per_pint
total_profit_adjusted = profit_per_pint_adjusted * pints_purchased

# Create results dataframe
check_df = pd.DataFrame({
    'best_profit': [best_profit],
    'one_std_profit': [total_profit_adjusted]
})

# Save to file
check_df.to_csv('check.tsv', sep='\t', index=False, float_format='%.2f')

Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.